This a detailed explanation of the code used to create the **Shopping Trends Dashboard** using **Streamlit**. This markdown will explain the key methods, their role, and how they contribute to the overall functionality of the dashboard.

---

# Shopping Trends Dashboard - Detailed Code Explanation

This document explains how the **Shopping Trends Dashboard** was developed using **Streamlit** for data visualization and interaction. It walks through the critical sections of the code, explaining the methods used, their purpose, and the flow of logic that allows the dashboard to function effectively.

## 1. Importing Required Libraries

```python
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st
from streamlit.components.v1 import html
from streamlit_option_menu import option_menu
import warnings
```

- **pandas**: Provides fast and flexible data manipulation tools, allowing us to load, clean, filter, and transform datasets.
- **numpy**: Used mainly for numerical operations, such as calculating means, sums, etc., although its usage is minimal in this context.
- **plotly.express**: A high-level interface for creating interactive visualizations such as bar charts, line charts, and maps.
- **streamlit**: The core library used to build the dashboard's user interface, handle interactions, and display visualizations.
- **streamlit_option_menu**: Provides an easy way to add a sidebar with multiple pages for navigation.
- **warnings**: Used to suppress unnecessary warnings, especially for warnings that may not affect the dashboard’s functionality, such as `FutureWarnings`.

## 2. Importing Custom Modules

```python
import home
import products
import locations
```

The dashboard is divided into three pages:
- **Home**: General overview of customer behavior.
- **Products**: Detailed analysis of products, their categories, sizes, and sales.
- **Locations**: Visualizes data specific to the geographic distribution of customers, sales, and subscriptions.

Each of these pages has its own module (`home.py`, `products.py`, `locations.py`) to encapsulate logic for filtering data, calculating metrics, and generating charts.

---

## 3. Dashboard Configuration

```python
def run():
    st.set_page_config(
        page_title="Shopping Trends",
        page_icon="🛍️",
        layout="wide"
    )
    warnings.simplefilter(action='ignore', category=FutureWarning)
```

- **st.set_page_config**: This function configures the page’s settings.
  - **page_title**: Sets the title of the browser tab.
  - **page_icon**: Sets an emoji icon in the tab.
  - **layout**: The `wide` option ensures that the dashboard takes full advantage of the screen width.

- **warnings.simplefilter**: Suppresses any `FutureWarning` that may appear due to changes in future versions of libraries. These warnings do not affect the functionality of the dashboard but may clutter the output.

---

## 4. Loading the Dataset

```python
@st.cache_data
def load_data(the_file_path):
    df = pd.read_csv(the_file_path)
    df.columns = df.columns.str.replace(" ", "_")
    df.rename(columns={"Purchase_Amount_(USD)": "Price_in_USD"}, inplace=True)
    df.set_index("Customer_ID", inplace=True)
    return df
```

### Explanation of Key Methods:

- **@st.cache_data**: This decorator caches the result of the `load_data` function, which means the data is only loaded and processed once. If the dataset doesn’t change, the cached version is used to improve performance.
  
- **load_data**:
  - **pd.read_csv(the_file_path)**: Loads the CSV file into a Pandas DataFrame.
  - **df.columns.str.replace(" ", "_")**: Replaces spaces in column names with underscores to make the column names easier to reference in code.
  - **df.rename()**: Renames the `Purchase_Amount_(USD)` column to `Price_in_USD` for better readability.
  - **df.set_index()**: Sets the `Customer_ID` column as the index, making it easier to reference customers for any aggregations or filtering.

### Benefits of Caching:
- Since loading and cleaning data can be a resource-heavy process, caching ensures that once data is processed, it doesn't have to be reloaded from the source file every time a page refreshes. This optimizes the dashboard's performance.

---

## 5. Applying Custom CSS Styles

```python
st.markdown(
    """
    <style>
         .main { text-align: center; }
         /* Custom CSS for specific elements */
    </style>
    """, unsafe_allow_html=True
)
```

- **st.markdown**: Allows us to inject HTML and CSS code to style the dashboard's elements, making it more visually appealing and user-friendly.
- The **CSS** included here adjusts the padding, text alignment, borders, hover effects, and font styles for specific components of the dashboard.

### Key CSS Styles:
- **.main**: Centers the main content on the page.
- **div.block-containers**: Adds padding around the content blocks to improve spacing and readability.

### Why Use Custom CSS?
- Streamlit's default styling can be quite basic. Custom CSS helps in creating a more polished, professional-looking dashboard.

---

## 6. Sidebar Navigation with `option_menu`

```python
with st.sidebar:
    page = option_menu(
        menu_title='Sidebar',
        options=['Home', 'Products', "Locations"],
        icons=['house-fill', 'person-circle', "map-fill"],
        menu_icon='chat-text-fill',
        default_index=0,
        styles={
            "container": {"padding": "5!important", "background-color": '#000'},
            "icon": {"color": "white", "font-size": "20px"},
            "nav-link-selected": {"background-color": "#5E0303"},
        }
    )
```

- **option_menu**: This method creates an interactive sidebar that allows users to navigate between different pages (Home, Products, Locations).
  - **options**: List of page names the user can switch between.
  - **icons**: Uses Bootstrap icons to visually represent each option.
  - **styles**: Custom styles for the sidebar. This controls colors, padding, and the appearance of selected items.

### Benefits:
- Having a sidebar allows users to switch between different views easily, enhancing navigation and user experience.

---

## 7. Page Content - Home Page

```python
if page == "Home":
    category_filter = st.multiselect("Select The Category 👕💎", ...)
    size_filter = st.multiselect("Select The Size 👔", ...)
    location_filter = st.selectbox("Select The Location 🌏", ...)
    
    with header:
        home.home_header()

    with content:
        df_filtered = home.filter_data(df, category_filter, size_filter, location_filter)
        
        # Display Key Metrics
        st.subheader(home.total_customers(df_filtered))
        st.subheader(home.average_rating(df_filtered))
        st.subheader(home.total_purchases(df_filtered))
        
        # Display Charts
        st.plotly_chart(home.create_category_chart(df_filtered), use_container_width=True)
        st.plotly_chart(home.create_gender_chart(df_filtered), use_container_width=True)
        st.plotly_chart(home.create_shipping_chart(df_filtered), use_container_width=True)
```

### Key Methods:
- **st.multiselect**: Allows users to select multiple values from a dropdown. These filters apply to categories, sizes, and locations.
  - **category_filter**: Filter the data by the selected product categories.
  - **size_filter**: Filter the data by the selected sizes.
  - **location_filter**: Filter by geographic location of the customers.
  
- **home.filter_data**: This method (defined in `home.py`) applies the filters to the dataset, returning a filtered DataFrame based on the selected options.

### Displaying Key Metrics:
- **st.subheader**: Displays key metrics for the Home page:
  - **home.total_customers(df_filtered)**: Calculates the total number of unique customers in the filtered data.
  - **home.average_rating(df_filtered)**: Shows the average rating from customer reviews.
  - **home.total_purchases(df_filtered)**: Displays the total number of purchases.

### Displaying Charts:
- **st.plotly_chart**: Renders the charts using Plotly:
  - **home.create_category_chart(df_filtered)**: A bar chart that breaks down the data by product category.
  - **home.create_gender_chart(df_filtered)**: A pie chart showing customer distribution by gender.
  - **home.create_shipping_chart(df_filtered)**: A line chart showing shipping data trends.

---

## 8. Products Page

```python
if page == "Products":
    # Similar structure to Home page with filters and metrics focused on products.
    category_filter = st.multiselect("Select The Category 👕💎", ...)
    size_filter = st.multiselect("Select The Size 👔", ...)
    location_filter = st.selectbox("Select The Location 🌏", ...)
    
    with header:
        products.products_header()
        
    with content:
        df_filtered = products.filter_data(df, category_filter, size_filter, location_filter)
        
        st.subheader(products.total_sales(df_filtered))
        st.subheader(products.number_of_category(df))
        st.subheader(products.number_of_products(df_filtered))
        
        st.plotly_chart(products.create_products_chart(df_filtered), use_container_width=True)
        st.plotly_chart(products.create_size_chart(df_filtered), use_container_width=True)
        st.plotly_chart(products.category_via_season_chart(df_filtered), use_container_width=True)
```

- This page is structured similarly to the Home page but focuses on product-related data.
- It includes the

 same types of filters (categories, sizes, and locations) and provides metrics specific to product sales and distribution.

---

## 9. Locations Page

```python
if page == "Locations":
    # Filters specific to Locations page
    category_filter = st.multiselect("Select The Category 👕💎", ...)
    size_filter = st.multiselect("Select The Size 👔", ...)
    season_filter = st.multiselect("Select The Season :snowflake::sunny:", ...)
    
    with header:
        locations.sales_header()
        
    with content:
        df_filtered = locations.filter_data(df, category_filter, size_filter, season_filter)
        
        # Visualizations specific to locations
        st.plotly_chart(locations.create_map(df_filtered), use_container_width=True)
        st.plotly_chart(locations.create_subscription_via_location(df_filtered), use_container_width=True)
        st.plotly_chart(locations.create_location_category(df_filtered), use_container_width=True)
        st.plotly_chart(locations.create_top3_review(df_filtered), use_container_width=True)
```

- **season_filter**: A new filter specific to the **Locations** page, allowing users to filter data based on the season of purchase.
- **locations.create_map**: Generates an interactive map that shows customer locations.
- **locations.create_subscription_via_location**: Displays data on subscription trends by location.
- **locations.create_top3_review**: Shows the top three customer reviews for each location.

---

## Conclusion

This **Shopping Trends Dashboard** demonstrates the power of **Streamlit** combined with **Pandas** for data manipulation and **Plotly** for interactive visualizations. The structure of the code is modular, with each page having its own logic encapsulated in separate modules (`home.py`, `products.py`, and `locations.py`). This separation of concerns makes the dashboard easier to maintain and extend.

The dashboard is designed with performance in mind, using **@st.cache_data** to optimize data loading and **CSS styling** to create a polished, professional look. The use of filters allows users to drill down into the data and explore trends in customer behavior, product sales, and geographic distribution interactively.